In [5]:
from keras.applications import VGG16

image_size = 224

vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

for layer in vgg_conv.layers[:-4]:
    layer.trainable = False

In [13]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()

#model.add(vgg_conv)
model.add(layers.Convolution2D(1, (4, 4), input_shape=(image_size, image_size, 3)))
model.add(layers.MaxPooling2D(pool_size=(2,2), strides=(20,20)))
model.add(layers.Flatten())
model.add(layers.Dense(1))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(50, activation='softmax'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 221, 221, 1)       49        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 1)         0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 121)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 122       
_________________________________________________________________
dropout_4 (Dropout)          (None, 1)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 50)                100       
Total params: 271
Trainable params: 271
Non-trainable params: 0
________________________________________________________

In [14]:
from keras.optimizers import Adam
adam = Adam(lr=0.0003)
model.compile(
    loss='mean_squared_error',
    metrics=['accuracy'],
    optimizer=adam
)

In [15]:
model.save('birds.hdf5')

In [4]:
data_dir = 'public_tests/00_test_img_input/train/images/'

In [5]:
import pandas as pd

df = pd.read_csv('public_tests/00_test_img_input/train/gt.csv').astype(str)
valdf = df.sample(frac=0.2, random_state=42)
traindf = pd.concat([df, valdf]).drop_duplicates(keep=False)

In [6]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='reflect'
)

generator_params = dict(
        directory=data_dir,
        x_col="filename",
        y_col="class_id",
        target_size=(image_size, image_size),
        batch_size=2,
        class_mode='categorical'
)

train_generator = datagen.flow_from_dataframe(
        dataframe=traindf,
        **generator_params
)

val_generator = datagen.flow_from_dataframe(
        dataframe=valdf,
        **generator_params
)

Found 2000 validated image filenames belonging to 50 classes.
Found 500 validated image filenames belonging to 50 classes.


In [7]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2,
        epochs=1,
        validation_data=val_generator,
        validation_steps=1
)

Epoch 1/1
2/2 [==============================] - 8s 4s/step - loss: 0.0207 - accuracy: 0.0000e+00 - val_loss: 0.0199 - val_accuracy: 0.0000e+00


In [16]:
from os.path import join

filenames = os.listdir('public_tests/00_test_img_input/test/123/')

test_generator = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True,
    rescale=1./255
).flow_from_dataframe(
    dataframe=pd.DataFrame(
        {'filename': filenames, 'class_id': '1'}
    ),
    directory=data_dir,
    x_col="filename",
    y_col="class_id",
    target_size=(image_size, image_size),
    batch_size=32
)

Found 8 validated image filenames belonging to 1 classes.


In [17]:
prediction = model.predict_generator(test_generator)

In [21]:
answers = prediction.argmax(axis=1)

{'0003.jpg': 45,
 '0001.jpg': 45,
 '0008.jpg': 31,
 '0007.jpg': 31,
 '0006.jpg': 45,
 '0000.jpg': 31,
 '0002.jpg': 31,
 '0009.jpg': 31}